In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image
import cv2 as cv
import os

# Read the frozen inference graph
with tf.gfile.FastGFile('cells_graph_3/frozen_inference_graph.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

# list paths of images to be boxed

names = []
image_paths = []
for path in os.listdir('test_images/'):
    if path.endswith('jpg'):
        names.append(path[:-4])
        image_paths.append('test_images/' + path)

# load images as PIL Image objects into list
images = []
images_cv = []
for path in image_paths:
    images.append(Image.open(path)) # image for bbox prediction
    images_cv.append(cv.imread(path)) # image to set rectangles on
    
# image = Image.open('test_images/image5.jpg')
# image1 = cv.imread('test_images/image5.jpg') # image to put rectangles on

Instructions for updating:
Use tf.gfile.GFile.


In [2]:
with tf.Session() as sess:
    # Restore session
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')

    def load_image_into_numpy_array(image):
        (im_width, im_height) = image.size
        print(im_width, im_height)
        if len(np.array(image.getdata()).shape) != 2:
            print('yay')
            return np.stack((image.getdata(),)*3, axis=-1).reshape((im_height, im_width, 3)).astype(np.uint8)
            # 
        else:
            return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
    
    for image, image1, name in zip(images, images_cv, names):
        os.mkdir('coordinates/{}'.format(name)) # make new folder for each image name
        text_file = open('coordinates/{}.txt'.format(name), 'w') # initate text file
        inp = load_image_into_numpy_array(image)

        # Run the model
        out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                        sess.graph.get_tensor_by_name('detection_scores:0'),
                        sess.graph.get_tensor_by_name('detection_boxes:0'),
                        sess.graph.get_tensor_by_name('detection_classes:0')],
                       feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})

        # Set up plotting of bounding boxes.
        # cropped_imgs = [] # coordinates of boxes in list for individidual boxes
        num_detections = int(out[0][0])
        for i in range(num_detections):
            classId = int(out[3][0][i])
            score = float(out[1][0][i])
            bbox = [float(v) for v in out[2][0][i]]
            if score > 0.2:
                x = bbox[1] * image.size[1]
                y = bbox[0] * image.size[0]
                right = bbox[3] * image.size[1]
                bottom = bbox[2] * image.size[0]
                # cropped_imgs.append([int(x), int(y), int(right), int(bottom)])
                cropped_image = [str(int(x)), str(int(y)),
                                 str(int(right)), str(int(bottom))]
                text_file.write(str(i) + ' ')
                for coord in cropped_image:
                    text_file.write(coord + ' ')
                text_file.write('\n')
                text_file_1 = open('coordinates/{}/{}.txt'.format(name,i), 'w')
                text_file_1.write('{}\n'.format(i))
                text_file_1.write('{}, {}'.format(int(bottom - y), int(right - x)))
                crop = image.crop(tuple([int(x), int(y), int(right), int(bottom)]))
                new_im = Image.new("L", (256, 256))
                new_im.paste(crop)
                new_im.save('coordinates/{}/{}.jpg'.format(name, i))
                # print(type(crop))
                # crops.append(crop)
                cv.rectangle(image1, (int(x), int(y)), (int(right), int(bottom)), (125, 255, 51), thickness=2)
                
        text_file.close()
        cv.imshow('TensorFlow Prediction', image1)
        cv.waitKey(0)
        cv.destroyAllWindows()


1024 1024
yay
1024 1024
yay
1024 1024
yay
1024 1024
yay
1024 1024
yay


In [ ]:
cropped_imgs

In [ ]:
image.size

In [ ]:
cv.imshow('TensorFlow Prediction', image1)